# VQLS with CUDA-Q

Varitional Quantum Linear Solver (VQLS) is hybrid quantum-classical algorithm for solving linear system of equations proposed by Bravo-Prieto et al [1].
$$Ax = b$$

VQLS prepares state $A|x\rangle$ and computes its overlap with $|b\rangle$.
State $|x\rangle = V(\theta)|0\rangle$ is prepared using a variational ansatz parameterized by the angles $\theta$.

VQLS cost function computes the overlap of $A|x\rangle$ and $|b\rangle$ as follows:
$$C_G = 1 - \frac{|\langle b|\psi \rangle|^2}{\langle \psi|\psi \rangle}$$

where $|\psi\rangle = A|x\rangle = A V(\theta)|0\rangle$

In this notebook we demonstrate VQLS with cudaq in python. 




[1] Bravo-Prieto, C., LaRose, R., Cerezo, M., Subasi, Y., Cincio, L., & Coles, P. J. (2023). Variational quantum linear solver. Quantum, 7, 1188.

In [9]:
# Necessary Imports
import cudaq
import numpy as np
from itertools import product
from cudaq import spin

### Ansatz
To variationally encode the $|x\rangle$ state we will use the hardware efficient ansatz as defined in the paper.

In [12]:
@cudaq.kernel
def ansatz(q: cudaq.qview, n_qubits: int, n_layers: int, theta: list[float]):
    for i in range(n_layers):
        for j in range(n_qubits):
            ry(theta[i * n_qubits + j], q[j])
        for j in range(n_qubits - 1):
            if ((i + j) % 2) == 0:
                cx(q[j], q[j + 1])

### Overlap test
Lets calculate the overlap of state $|b\rangle$ and $A|x\rangle$ given by $|\langle b|\psi \rangle|^2$.

This is the numerator of $C_G$. We use the overlap test as described in [2].

[2] Patil, H., Wang, Y., & Krstić, P. S. (2022). Variational quantum linear solver with a dynamic ansatz. Physical Review A, 105(1), 012423.

In [13]:


swap_matrix = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1]
])

# Register custom SWAP operation
cudaq.register_operation("custom_swap", swap_matrix)


@cudaq.kernel
def overlap_bpsi(A: list[int], n_qubits: int, n_layers: int, theta: list[float]):
    q = cudaq.qvector(2*n_qubits+1)
    ansatz(q, n_qubits, n_layers, theta)
    for i in range(n_qubits, 2*n_qubits):
        h(q[i])
    i = 0
    h(q[2*n_qubits])
    for pauli in A:
        if pauli == 1:
            x.ctrl(q[2*n_qubits], q[i])
        elif pauli == 2:
            y.ctrl(q[2*n_qubits], q[i])
        elif pauli == 3:
            z.ctrl(q[2*n_qubits], q[i])
        i += 1
    for i in range(n_qubits):
        custom_swap.ctrl(q[2*n_qubits], q[n_qubits+i], q[i])
    h(q[2*n_qubits])
    mz(q[2*n_qubits])

print(cudaq.draw(overlap_bpsi, [3,0,3, 1, 0,3], 3, 2, [0.1]*6))

     ╭─────────╮           ╭─────────╮     ╭───╮               ╭─────────────╮»
q0 : ┤ ry(0.1) ├─────●─────┤ ry(0.1) ├─────┤ z ├───────────────┤>            ├»
     ├─────────┤   ╭─┴─╮   ├─────────┤     ╰─┬─╯               │             │»
q1 : ┤ ry(0.1) ├───┤ x ├───┤ ry(0.1) ├──●────┼─────────────────│             │»
     ├─────────┤╭──┴───┴──╮╰─────────╯╭─┴─╮  │  ╭───╮          │ custom_swap │»
q2 : ┤ ry(0.1) ├┤ ry(0.1) ├───────────┤ x ├──┼──┤ z ├──────────│             │»
     ╰──┬───┬──╯╰─────────╯           ╰───╯  │  ╰─┬─╯╭───╮     │             │»
q3 : ───┤ h ├────────────────────────────────┼────┼──┤ x ├─────┤>            ├»
        ├───┤                                │    │  ╰─┬─╯     ╰──────┬──────╯»
q4 : ───┤ h ├────────────────────────────────┼────┼────┼──────────────┼───────»
        ├───┤                                │    │    │  ╭───╮       │       »
q5 : ───┤ h ├────────────────────────────────┼────┼────┼──┤ z ├───────┼───────»
        ├───┤                           

### Hadamard Test for the denominator of the cost function

To calculate the denominator of the VQLS cost function we use the Hadamard Test.


In [15]:
@cudaq.kernel
def psi_psi(A: list[int], n_qubits: int, n_layers: int, theta: list[float]):
    q = cudaq.qvector(n_qubits+1)
    ansatz(q, n_qubits, n_layers, theta)
    h(q[n_qubits])
    i = 0
    for pauli in A:
        if pauli == 1:
            x.ctrl(q[n_qubits], q[i])
        elif pauli == 2:
            y.ctrl(q[n_qubits], q[i])
        elif pauli == 3:
            z.ctrl(q[n_qubits], q[i])
        i += 1
        if (i == (n_qubits)):
            i = 0
    h(q[n_qubits])
    mz(q[n_qubits])

    
print(cudaq.draw(psi_psi, [3,0,3, 1, 0,3], 3, 2, [0.1]*6))

     ╭─────────╮           ╭─────────╮     ╭───╮     ╭───╮          
q0 : ┤ ry(0.1) ├─────●─────┤ ry(0.1) ├─────┤ z ├─────┤ x ├──────────
     ├─────────┤   ╭─┴─╮   ├─────────┤     ╰─┬─╯     ╰─┬─╯          
q1 : ┤ ry(0.1) ├───┤ x ├───┤ ry(0.1) ├──●────┼─────────┼────────────
     ├─────────┤╭──┴───┴──╮╰─────────╯╭─┴─╮  │  ╭───╮  │  ╭───╮     
q2 : ┤ ry(0.1) ├┤ ry(0.1) ├───────────┤ x ├──┼──┤ z ├──┼──┤ z ├─────
     ╰──┬───┬──╯╰─────────╯           ╰───╯  │  ╰─┬─╯  │  ╰─┬─╯╭───╮
q3 : ───┤ h ├────────────────────────────────●────●────●────●──┤ h ├
        ╰───╯                                                  ╰───╯



     ╭─────────╮     ╭─────────╮╭───╮          
q0 : ┤ ry(0.1) ├──●──┤ ry(0.1) ├┤ z ├──────────
     ├─────────┤╭─┴─╮├─────────┤╰─┬─╯╭───╮     
q1 : ┤ ry(0.1) ├┤ x ├┤ ry(0.1) ├──┼──┤ z ├─────
     ╰──┬───┬──╯╰───╯╰─────────╯  │  ╰─┬─╯╭───╮
q2 : ───┤ h ├─────────────────────●────●──┤ h ├
        ╰───╯                             ╰───╯

